In [111]:
import os
import pandas as pd
from mistralai import Mistral
from supabase import create_client, Client
import pymupdf
from tqdm import tqdm
from io import BufferedReader

In [115]:
# For storing the stuff
supabase_url: str = os.environ.get("SUPABASE_URL")
supabase_key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

auth_response = supabase.auth.sign_in_with_password({"email": "mister.x.vic@googlemail.com", "password": "FG23xi_i!"})

In [121]:
# To read the text and create embedding
mistral_api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=mistral_api_key)

## Upsert companies

In [53]:
industry_sector = pd.read_csv("sasb-industry-sector.csv")

In [52]:
companies = (
    pd.read_csv("https://docs.google.com/spreadsheets/d/1Nlyf8Yz_9Fst8rEmQc2IMc-DWLF1fpmBTB7n4FlZwxs/export?format=csv&gid=0", skiprows=2)
    .query("verified == 'yes'")
    .rename(columns={
        "company": "name",
        "SASB industry \n(SICS® Industries)": "industry",
        "start PDF": "startPDF",
        "end PDF": "endPDF",
    })
    .dropna()
    .loc[:, ['name', 'isin', 'country', 'industry', "startPDF", "endPDF"]]
)

companies

,name,isin,country,industry,startPDF,endPDF
0,Netcompany,DK0060952919,Denmark,Internet Media & Services,62.0,154.0
1,Tryg,DK0060636678,Denmark,Insurance,53.0,131.0
2,DSV,DK0060079531,Denmark,Air Freight & Logistics,42.0,84.0
3,Lundbeck,DK0061804697,Denmark,Biotechnology & Pharmaceuticals,59.0,144.0
4,Vestas,DK0061539921,Denmark,Wind Technology & Project Developers,51.0,132.0
...,...,...,...,...,...,...
172,RWE,DE0007037129,Germany,Electric Utilities & Power Generators,74.0,186.0
173,Triodos Bank,n.a.,Netherlands,Commercial Banks,167.0,327.0
174,AXA SA,FR0000120628,France,Insurance,129.0,271.0
175,Teleperformance,FR0000051807,France,Software & IT Services,93.0,204.0


In [55]:
for _, company in companies.iterrows():
    companyUpsert_response = (
        supabase.table("companies")
        .upsert(
            {
                "name": company["name"], 
                "isin": company["isin"], 
                "country": company["country"],
                "industry": company["industry"],
                "sector": industry_sector[industry_sector.industry == company["industry"]] ['sector'].values[0],
            }, 
            on_conflict=["name"]
        )
        .execute()
    )

    print(companyUpsert_response)

data=[{'id': '84f555f7-0dfe-4287-9ebb-4a8510851960', 'name': 'Netcompany', 'isin': 'DK0060952919', 'created_at': '2025-03-20T22:08:02.596329+00:00', 'country': 'Denmark', 'sector': 'Technology & Communications', 'industry': 'Internet Media & Services'}] count=None
data=[{'id': 'dfd1ae0e-fdd2-402b-87b7-8269fc136bf1', 'name': 'Tryg', 'isin': 'DK0060636678', 'created_at': '2025-03-20T22:08:02.858821+00:00', 'country': 'Denmark', 'sector': 'Financials', 'industry': 'Insurance'}] count=None
data=[{'id': '2697bd05-bcc6-46c6-959f-b28c9f79601d', 'name': 'DSV', 'isin': 'DK0060079531', 'created_at': '2025-03-20T22:08:03.236784+00:00', 'country': 'Denmark', 'sector': 'Transportation', 'industry': 'Air Freight & Logistics'}] count=None
data=[{'id': 'c403ab7e-9ab8-4a2d-886f-32f65885cba2', 'name': 'Lundbeck', 'isin': 'DK0061804697', 'created_at': '2025-03-20T22:08:03.606235+00:00', 'country': 'Denmark', 'sector': 'Health Care', 'industry': 'Biotechnology & Pharmaceuticals'}] count=None
data=[{'id': 

In [89]:
reports_dir = "/Users/victor/Library/CloudStorage/Dropbox/5 - Sustainability Reporting Navigator/1-outreach/2025-03 early csrd analysis/data/pdf/"
avlbl_reports = os.listdir(reports_dir)

In [86]:
# Create the hypothetical filename the company from the Google Sheet would have
companies = (
    companies
    .assign(
        isin_or_name = lambda x: [isin if isin != "n.a." else name for name, isin in zip(x["name"], x["isin"])], 
        filename=lambda x: x["isin_or_name"] + "_2024_AR.pdf"
        )
    # add ID from supabase
    .merge(
        pd.DataFrame(supabase.table("companies").select("id, isin, name").execute().data),
        left_on=["name", "isin"],
        right_on=["name", "isin"]
    )
)

companies

,name,isin,country,industry,startPDF,endPDF,filename,isin_or_name,id
0,Netcompany,DK0060952919,Denmark,Internet Media & Services,62.0,154.0,DK0060952919_2024_AR.pdf,DK0060952919,84f555f7-0dfe-4287-9ebb-4a8510851960
1,Tryg,DK0060636678,Denmark,Insurance,53.0,131.0,DK0060636678_2024_AR.pdf,DK0060636678,dfd1ae0e-fdd2-402b-87b7-8269fc136bf1
2,DSV,DK0060079531,Denmark,Air Freight & Logistics,42.0,84.0,DK0060079531_2024_AR.pdf,DK0060079531,2697bd05-bcc6-46c6-959f-b28c9f79601d
3,Lundbeck,DK0061804697,Denmark,Biotechnology & Pharmaceuticals,59.0,144.0,DK0061804697_2024_AR.pdf,DK0061804697,c403ab7e-9ab8-4a2d-886f-32f65885cba2
4,Vestas,DK0061539921,Denmark,Wind Technology & Project Developers,51.0,132.0,DK0061539921_2024_AR.pdf,DK0061539921,ef6facde-60b5-49d6-8b51-c500ec0632a0
...,...,...,...,...,...,...,...,...,...
172,RWE,DE0007037129,Germany,Electric Utilities & Power Generators,74.0,186.0,DE0007037129_2024_AR.pdf,DE0007037129,e1202787-9274-424c-9ea6-a4ecfe8c3e3c
173,Triodos Bank,n.a.,Netherlands,Commercial Banks,167.0,327.0,Triodos Bank_2024_AR.pdf,Triodos Bank,59788d0c-4fed-41c5-808b-f7bd5ef4ed5e
174,AXA SA,FR0000120628,France,Insurance,129.0,271.0,FR0000120628_2024_AR.pdf,FR0000120628,c6f3a472-6fa9-4724-93fc-c5afc2c54c5f
175,Teleperformance,FR0000051807,France,Software & IT Services,93.0,204.0,FR0000051807_2024_AR.pdf,FR0000051807,3b8c223c-c1c3-4e72-80c7-f40825e8b08f


In [112]:
from tusclient import client

In [116]:
def upload_file_to_supabase(
    supabase_url:str, file_name: str, file: BufferedReader, access_token: str
):
    # create Tus client
    my_client = client.TusClient(
        f"{supabase_url}/storage/v1/upload/resumable",
        headers={"Authorization": f"Bearer {access_token}", "x-upsert": "true"},
    )
    uploader = my_client.uploader(
        file_stream=file,
        chunk_size=(6 * 1024 * 1024),
        metadata={
            "bucketName": "document-pdfs",
            "objectName": file_name,
            "contentType": "application/pdf",
            "cacheControl": "3600",
        },
    )
    uploader.upload()

In [120]:
for _, company in tqdm(companies.iterrows(), total=len(companies)):
    try:
        if company["filename"] in avlbl_reports:
            documentUpsert_response = (
                supabase.table("documents")
                .upsert(
                    {
                        "company_id": company["id"],
                        "year": 2024,
                        "type": "Annual Report",
                        "pages": f"({int(company.startPDF)}, {int(company.endPDF)})"
                    },
                    on_conflict=["company_id, year, type"]
                )
                .execute()
            )

            document_id = documentUpsert_response.data[0].get("id")

            doc = pymupdf.open(reports_dir+company["filename"])
            doc.select(list(range(int(company.startPDF) - 1, min(int(company.endPDF), len(doc)))))
            cropped_path = reports_dir.replace("data/pdf/", "data/pdf-cropped/")+document_id+".pdf"
            doc.save(cropped_path)

            with open(cropped_path, "rb") as fs:
                upload_file_to_supabase(
                    supabase_url=supabase_url,
                    file_name=document_id+".pdf",
                    file=fs,
                    access_token=auth_response.session.access_token,
                )

    except Exception as e:
        print(company.name, e)

  1%|          | 1/177 [00:00<02:35,  1.13it/s]

0 Attempt to retrieve create file url with status 403


  1%|          | 2/177 [00:01<02:15,  1.29it/s]

1 Attempt to retrieve create file url with status 403


  2%|▏         | 3/177 [00:02<02:26,  1.18it/s]

...

 97%|█████████▋| 172/177 [47:22<01:05, 13.16s/it]

171 Failed to open file '/Users/victor/Library/CloudStorage/Dropbox/5 - Sustainability Reporting Navigator/1-outreach/2025-03 early csrd analysis/data/pdf/n.a_2024_AR.pdf'.


 98%|█████████▊| 173/177 [47:22<00:39,  9.99s/it]

172 Failed to open file '/Users/victor/Library/CloudStorage/Dropbox/5 - Sustainability Reporting Navigator/1-outreach/2025-03 early csrd analysis/data/pdf/DE0007037129_2024_AR.pdf'.


100%|██████████| 177/177 [48:42<00:00, 16.51s/it]


In [125]:
documents = pd.DataFrame(supabase.table("documents").select("*").execute().data)
documents

,id,company_id,year,created_at,type,pages
0,efdcfa4d-d1be-4cee-99bc-edc44bd65312,84f555f7-0dfe-4287-9ebb-4a8510851960,2024,2025-03-20T22:39:06.628541+00:00,Annual Report,"(62, 154)"
1,aa6b5063-5095-46b9-b136-820da4ececa1,dfd1ae0e-fdd2-402b-87b7-8269fc136bf1,2024,2025-03-20T22:42:31.849655+00:00,Annual Report,"(53, 131)"
2,476d5914-34e7-4aa9-807c-ce9e2d5b5550,2697bd05-bcc6-46c6-959f-b28c9f79601d,2024,2025-03-20T22:42:53.343695+00:00,Annual Report,"(42, 84)"
3,fc84bde4-9bb0-4346-b4c0-ba2f4f62bf2c,c403ab7e-9ab8-4a2d-886f-32f65885cba2,2024,2025-03-20T22:44:25.645413+00:00,Annual Report,"(59, 144)"
4,ef045bd2-d948-4f96-a125-9eb79e7632ac,ef6facde-60b5-49d6-8b51-c500ec0632a0,2024,2025-03-20T22:44:34.114273+00:00,Annual Report,"(51, 132)"
...,...,...,...,...,...,...
170,01231b03-e9fb-424c-9a18-2701e247adb0,504f3d59-cbb9-4c87-80d4-cbd905f407bb,2024,2025-03-20T23:38:25.414525+00:00,Annual Report,"(374, 598)"
171,d85d9632-d78a-4a93-96f6-260141c13f39,e1202787-9274-424c-9ea6-a4ecfe8c3e3c,2024,2025-03-20T23:38:25.818417+00:00,Annual Report,"(74, 186)"
172,b0ab655e-83e8-418f-b7e7-bab737a7e5ba,59788d0c-4fed-41c5-808b-f7bd5ef4ed5e,2024,2025-03-20T23:38:26.184884+00:00,Annual Report,"(167, 327)"
173,0346dbfe-6ca9-4342-85b1-a11078ca4435,c6f3a472-6fa9-4724-93fc-c5afc2c54c5f,2024,2025-03-20T23:39:17.060021+00:00,Annual Report,"(129, 271)"


In [130]:
companies = (
    companies
    .rename(
        columns={'id': 'company_id'}
    )
    .merge(
        (
            documents
            .loc[:, ['id', 'company_id']]
            .rename(
                columns={'id': 'document_id'}
            )
        ), 
        on='company_id'
    )
)

In [137]:
# Use Mistral OCR to read text and embeddings and upload them
cropped_dir = "/Users/victor/Library/CloudStorage/Dropbox/5 - Sustainability Reporting Navigator/1-outreach/2025-03 early csrd analysis/data/pdf-cropped/"
for _, company in tqdm(companies.iterrows(), total=len(companies)):
    if company["document_id"] in [x.replace(".pdf", "") for x in os.listdir(cropped_dir)]:
        try:
            cropped_path = cropped_dir + company['document_id'] + ".pdf"

            with open(cropped_path, "rb") as f:
                uploaded_pdf = client.files.upload(
                    file={
                        "file_name": cropped_path,
                        "content": f,
                    },
                    purpose="ocr"
                )
            
            signed_url = client.files.get_signed_url(file_id=uploaded_pdf.id)

            ocr_response = client.ocr.process(
                model="mistral-ocr-latest",
                document={
                    "type": "document_url",
                    "document_url": signed_url.url,
                }
            )

            for n, page in enumerate(ocr_response.pages):
                embeddings_batch_response = client.embeddings.create(
                    model="mistral-embed",
                    inputs=page.markdown,
                )

                pageInsert_response = (
                    supabase.table("pages")
                    .upsert(
                        {
                            "document_id": company["document_id"],
                            "page": int(company.startPDF) + n,
                            "content": page.markdown,
                            "embedding": embeddings_batch_response.data[0].embedding
                        }
                    )
                    .execute()
                )

        except Exception as e:
            print(company["name"], e)

  4%|▍         | 7/175 [06:20<2:32:57, 54.63s/it]

Pandora API error occurred: Status 400
{"object":"error","message":"File https://mistralaifilesapiprodswe.blob.core.windows.net/fine-tune/2bc5c3f8-3b5d-4f02-a4bb-e694f88d04e is too large. Maximum allowed size is 52.4 MB bytes.","type":"invalid_file","param":null,"code":"1901"}


  8%|▊         | 14/175 [14:13<2:57:46, 66.25s/it]

Ørsted API error occurred: Status 400
{"object":"error","message":"Input is too long. Max length is 8192 got 12988","type":"invalid_request_error","param":null,"code":null}


 10%|▉         | 17/175 [17:19<2:46:01, 63.05s/it]

...

 93%|█████████▎| 162/175 [3:45:04<16:26, 75.88s/it] 

Sitowise Group Plc API error occurred: Status 400
{"object":"error","message":"Input is too long. Max length is 8192 got 14015","type":"invalid_request_error","param":null,"code":null}


 96%|█████████▌| 168/175 [3:53:15<09:33, 81.95s/it]

NWB Bank API error occurred: Status 400
{"object":"error","message":"Input is too long. Max length is 8192 got 8877","type":"invalid_request_error","param":null,"code":null}


100%|██████████| 175/175 [4:03:50<00:00, 83.60s/it] 
